In [45]:
# encryption

In [10]:
import cv2
import numpy as np
import random
import os
import time

start=time.time();
def generate_keys(height, width, a):
    random.seed(42)
    
    r = 3.68
    b = 0.012
    a = 0.011
    
    num_len = height * width * 3
    key_list = []

    for i in range(num_len // 3):
        x = random.randint(0, 255)
        y = random.randint(0, 255)
        z = random.randint(0, 255)
        
        num2 = (r * y * (1 - y)) + (b * y * (z * 2)) + (a * (x * 2))
        num3 = (r * z * (1 - z)) + (b * z * (x * 2)) + (a * (y * 2))
        num1 = (r * x * (1 - x)) + (b * x * (y * 2)) + (a * (z * 2))
        
        key_list.extend([abs(num1), abs(num2), abs(num3)])
        
    num = np.array(key_list, dtype=np.uint8).reshape((height, width, 3))

    np.save("updated_key.npy", num)
    return num

def rearrange_video_frames(video_path, output_folder, height, width, a):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    frame_list = []

    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    print(f"Video properties: Frame Count={frame_count}, Width={width}, Height={height}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_list.append(frame)

    cap.release()

    # Generate keys and use them for consistent shuffling
    keys = generate_keys(height, width, a)
    random.seed(42)
    frame_indices = list(range(frame_count))
    random.shuffle(frame_indices)

    # Save the shuffled indices to a file
    with open("shuffled.txt", "w") as file:
        for idx in frame_indices:
            file.write(f"{idx}\n")

    for i, idx in enumerate(frame_indices):
        output_frame_path = os.path.join(output_folder, f"frame_{i:04d}.png")
        cv2.imwrite(output_frame_path, frame_list[idx])
        # print(f"Saved frame {idx} as {output_frame_path}")

    print("Rearranging of video frames completed.")

video_path = r"C:\Users\Asus\PYTHON\Images\vid1.avi"
output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino0"
height = 240;  
width = 320;  
a= 0.011 
rearrange_video_frames(video_path, output_folder, height, width, a)

# Initialize Vigenère cipher parameters
m = 256
n = 256

# Generate Vigenère cipher lookup table
vigenere_table = np.zeros((m, n), dtype=np.uint8)
for i in range(m):
    for j in range(n):
        vigenere_table[i, j] = (i + j) % 256

# Path to the input frames
input_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino0"

# Path to the output folder
output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino1"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate over frames in the input folder
for frame_file in os.listdir(input_folder):
    frame_path = os.path.join(input_folder, frame_file)
    
    # Read the frame
    frame = cv2.imread(frame_path)
    
    if frame is None:
        print(f"Error reading frame {frame_file}")
        continue
    
    # Extract the frame index from the filename
    frame_index = int(frame_file.split("_")[-1].split(".")[0])
    
    # Apply Vigenère cipher to each channel of the frame
    plane1, plane2, plane3 = cv2.split(frame)
    plane1 = vigenere_table[frame_index % m, plane1]
    plane2 = vigenere_table[frame_index % m, plane2]
    plane3 = vigenere_table[frame_index % m, plane3]

    # Merge the channels back into a frame
    frame = cv2.merge([plane1, plane2, plane3])

    # Save the modified frame to the output folder
    output_path = os.path.join(output_folder, frame_file)
    cv2.imwrite(output_path, frame)
    

def shuffle_pixels_of_frames(input_folder, output_folder, key_file, indices_file):
    # Load the pre-generated keys
    keys = np.load(key_file)
    
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Create a dictionary to save the indices mapping
    indices_mapping = {}

    # Iterate over frames in the input folder
    for frame_file in os.listdir(input_folder):
        frame_path = os.path.join(input_folder, frame_file)
        
        # Read the frame
        frame = cv2.imread(frame_path)
        
        if frame is None:
            print(f"Error reading frame {frame_file}")
            continue
        
        # Extract the frame index from the filename
        frame_index = int(frame_file.split("_")[-1].split(".")[0])
        
        # Get the height and width of the frame
        height, width, _ = frame.shape
        
        # Ensure keys match the frame size
        if keys.shape[0] != height or keys.shape[1] != width:
            print(f"Error: Key dimensions do not match frame dimensions for {frame_file}")
            continue
        
        # Create an empty array to hold the shuffled frame
        shuffled_frame = np.zeros_like(frame)
        
        # Create indices grid
        row_indices, col_indices = np.indices((height, width))
        
        # Calculate new positions
        new_row_indices = (row_indices + keys[..., 0]) % height
        new_col_indices = (col_indices + keys[..., 1]) % width
        
        # Use advanced indexing to shuffle the frame
        shuffled_frame[new_row_indices, new_col_indices] = frame[row_indices, col_indices]
        
        # Save the shuffled frame to the output folder
        output_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_path, shuffled_frame)
        # print(f"Saved shuffled frame {frame_file} as {output_path}")
        
        # Save the indices mapping
        indices_mapping[frame_file] = (new_row_indices, new_col_indices)
    
    # Save the indices mapping to a file
    np.save(indices_file, indices_mapping)
    
    print("Shuffling of pixels in frames completed.")

# Paths and parameters
input_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino1"  # Path to the original frames
output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino2"  # Path to save shuffled frames
key_file = "updated_key.npy"  # Path to the pre-generated keys file
indices_file = "shuffled.npy"  # Path to save the shuffled indices

# Shuffle the pixels of the frames
shuffle_pixels_of_frames(input_folder, output_folder, key_file, indices_file)

def xor_images(input_folder, output_folder, key_file):
    # Load the key
    key = np.load(key_file)
    
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get sorted list of frame files
    frame_files = sorted(os.listdir(input_folder))
    
    if not frame_files:
        print("No frames found in the input folder.")
        return

    
    for i, frame_file in enumerate(frame_files):
        frame_path = os.path.join(input_folder, frame_file)
        
        # Read the frame
        frame = cv2.imread(frame_path)
        
        if frame is None:
            print(f"Error reading frame {frame_file}")
            continue
        
        # Ensure key matches the frame size
        if key.shape != frame.shape:
            print(f"Error: Key dimensions do not match frame dimensions for {frame_file}")
            continue
        
        # XOR the current frame with the key
        xored_frame = cv2.bitwise_xor(frame, key)
        
        # Save the XORed frame
        output_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_path, xored_frame)

        # print(f"Saved XORed frame {frame_file} as {output_path}")
        
    
    print("XOR operation on images completed.")
# Paths and parameters
input_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino2"  # Path to the input frames
output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino3"  # Path to save the XORed frames
key_file = "updated_key.npy"  # Path to the key file

# Perform XOR operation on the images
xor_images(input_folder, output_folder, key_file)
end = time.time()
print("Time taken", end - start, "s")

Video properties: Frame Count=114, Width=320, Height=240
Rearranging of video frames completed.
Shuffling of pixels in frames completed.
XOR operation on images completed.
Time taken 8.546785116195679 s


In [47]:
# decryption

In [11]:
import glob

start=time.time();
def decrypt_images(input_folder, output_folder, key_file):
    # Load the key
    key = np.load(key_file)
    
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get sorted list of frame files
    frame_files = sorted(os.listdir(input_folder))
    
    if not frame_files:
        print("No frames found in the input folder.")
        return

    
    for i, frame_file in enumerate(frame_files):
        frame_path = os.path.join(input_folder, frame_file)
        
        # Read the XORed frame
        xored_frame = cv2.imread(frame_path)
        
        if xored_frame is None:
            print(f"Error reading frame {frame_file}")
            continue
        
        # Ensure key matches the frame size
        if key.shape != xored_frame.shape:
            print(f"Error: Key dimensions do not match frame dimensions for {frame_file}")
            continue
        
        # XOR the current XORed frame with the key to get the original frame
        original_frame = cv2.bitwise_xor(xored_frame, key)
        
        # Save the original frame
        output_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_path, original_frame)

        # print(f"Saved original frame {frame_file} as {output_path}")
        
    
    print("Decryption operation on images completed.")

# Perform XOR operation on the images
xor_images(input_folder, output_folder, key_file)

# Paths for decryption
encrypted_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino3"  # Path to the encrypted frames
decrypted_output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino4"  # Path to save the decrypted frames

# Perform decryption operation on the images
decrypt_images(encrypted_folder, decrypted_output_folder, key_file)

def unshuffle_pixels_of_frames(input_folder, output_folder, indices_file):
    # Load the saved indices mapping
    indices_mapping = np.load(indices_file, allow_pickle=True).item()
    
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over frames in the input folder
    for frame_file in os.listdir(input_folder):
        frame_path = os.path.join(input_folder, frame_file)
        
        # Read the frame
        frame = cv2.imread(frame_path)
        
        if frame is None:
            print(f"Error reading frame {frame_file}")
            continue
        
        # Get the height and width of the frame
        height, width, _ = frame.shape
        
        # Create an empty array to hold the unshuffled frame
        unshuffled_frame = np.zeros_like(frame)
        
        # Get the saved indices for the current frame
        new_row_indices, new_col_indices = indices_mapping[frame_file]
        
        # Create indices grid
        row_indices, col_indices = np.indices((height, width))
        
        # Use advanced indexing to unshuffle the frame
        unshuffled_frame[row_indices, col_indices] = frame[new_row_indices, new_col_indices]
        
        # Save the unshuffled frame to the output folder
        output_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_path, unshuffled_frame)
        # print(f"Saved unshuffled frame {frame_file} as {output_path}")
    
    print("Unshuffling of pixels in frames completed.")

# Paths and parameters
input_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino4"  # Path to the shuffled frames
output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino5"  # Path to save unshuffled frames
indices_file = "shuffled.npy"  # Path to the saved shuffled indices

# Unshuffle the pixels of the frames
unshuffle_pixels_of_frames(input_folder, output_folder, indices_file)


# Initialize Vigenère cipher parameters
m = 256
n = 256

# Generate Vigenère cipher lookup table
vigenere_table = np.zeros((m, n), dtype=np.uint8)
for i in range(m):
    for j in range(n):
        vigenere_table[i, j] = (i + j) % 256

# Path to the input encrypted frames
input_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino1"

# Path to the output folder for decrypted frames
output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino6"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate over encrypted frames in the input folder
for frame_file in os.listdir(input_folder):
    frame_path = os.path.join(input_folder, frame_file)
    
    # Read the encrypted frame
    frame = cv2.imread(frame_path)
    
    if frame is None:
        print(f"Error reading frame {frame_file}")
        continue
    
    # Extract the frame index from the filename
    frame_index = int(frame_file.split("_")[-1].split(".")[0])
    
    # Apply reverse Vigenère cipher to each channel of the frame
    plane1, plane2, plane3 = cv2.split(frame)
    plane1 = vigenere_table[(256 - frame_index) % m, plane1]
    plane2 = vigenere_table[(256 - frame_index) % m, plane2]
    plane3 = vigenere_table[(256 - frame_index) % m, plane3]

    # Merge the channels back into a frame
    frame = cv2.merge([plane1, plane2, plane3])

    # Save the decrypted frame to the output folder
    output_path = os.path.join(output_folder, frame_file)
    cv2.imwrite(output_path, frame);
    
def revert_rearranged_frames(input_folder, output_folder, indices_file):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read the shuffled indices from the file
    with open(indices_file, "r") as file:
        shuffled = [int(line.strip()) for line in file.readlines()]

    frame_files = sorted(glob.glob(os.path.join(input_folder, "*.png")))

    if len(frame_files) != len(shuffled):
        print(f"Error: Expected {len(shuffled)} frames, but found {len(frame_files)}")
        return

    # Create a mapping from the shuffled index back to the original index
    index_map = {shuffled[i]: i for i in range(len(shuffled))}

    for original_idx in range(len(shuffled)):
        shuffled_idx = index_map[original_idx]
        input_frame_path = frame_files[shuffled_idx]
        output_frame_path = os.path.join(output_folder, f"frame_{original_idx:04d}.png")
        
        frame = cv2.imread(input_frame_path)
        if frame is None:
            print(f"Error: Cannot read frame {input_frame_path}")
            continue

        cv2.imwrite(output_frame_path, frame)
        # print(f"Saved frame {shuffled_idx} as {output_frame_path}")

    print("Reversion of video frames completed.")

input_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino6" 
output_folder = "C:\\Users\\Asus\\PYTHON\\Images\\rhino7" 
indices_file = "shuffled.txt" 

end=time.time();
revert_rearranged_frames(input_folder, output_folder, indices_file)
print("time taken",end-start,"s");

XOR operation on images completed.
Decryption operation on images completed.
Unshuffling of pixels in frames completed.
Reversion of video frames completed.
time taken 10.254845380783081 s


In [12]:
image1=cv2.imread(r"C:\Users\Asus\PYTHON\Images\f4\frame_0029.png");
image2=cv2.imread(r"C:\Users\Asus\PYTHON\Images\f0\frame_0029.png");
correlation=np.corrcoef(image1.flatten(),image2.flatten())[0,1]
print(correlation)

0.6638209439277277


In [62]:
# geometric attack-rotational

In [13]:
import cv2
import numpy as np
import os

def decrypt_image(encrypted_image, key):
    decrypted_image = (encrypted_image.astype(np.uint16) - key.astype(np.uint16))
    return decrypted_image.astype(np.uint8)

def perform_rotation_attack(input_folder, output_folder, key):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all images in the input folder
    for filename in os.listdir(input_folder):
        # Read the encrypted image
        encrypted_image_path = os.path.join(input_folder, filename)
        encrypted_image = cv2.imread(encrypted_image_path)
        
        if encrypted_image is None:
            print(f"Error reading image: {encrypted_image_path}")
            continue
        
        # Perform rotational attack (90 degrees clockwise)
        height, width = encrypted_image.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), 90, 1)
        rotated_image1 = cv2.warpAffine(encrypted_image, rotation_matrix, (width, height))
        
        # Perform rotational attack (90 degrees counterclockwise)
        rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), -90, 1)
        rotated_image2 = cv2.warpAffine(encrypted_image, rotation_matrix, (width, height))
        
        # Decrypt the images
        decrypted_image1 = decrypt_image(encrypted_image, rotated_image1)
        decrypted_image2 = decrypt_image(encrypted_image, rotated_image2)
        
        # Save the decrypted images
        output_path1 = os.path.join(output_folder, f"decrypted_rotated1_{filename}")
        output_path2 = os.path.join(output_folder, f"decrypted_rotated2_{filename}")
        cv2.imwrite(output_path1, decrypted_image1)
        cv2.imwrite(output_path2, decrypted_image2)

        # print(f"Decrypted and saved images: {output_path1}, {output_path2}")

# Paths
input_folder = r"C:\Users\Asus\PYTHON\Images\f3"  # Path to the input folder of encrypted images
output_folder = r"C:\Users\Asus\PYTHON\Images\Rotation"  # Path to save the decrypted images
key = np.load("updated_key.npy")  # Path to the key file

# Perform rotational attack and decryption on images in the input folder
perform_rotation_attack(input_folder, output_folder, key)


In [68]:
# BER on rotation attack

In [6]:
# anti-clockwise
def calculate_ber(received_image, original_image):
    # Calculate the Hamming distance between the two images
    hamming_distance = np.count_nonzero(received_image != original_image)

    # Calculate the number of pixels in the image
    num_pixels = received_image.size

    # Calculate the BER
    ber = hamming_distance / num_pixels

    return ber

# Load the original and received images
original_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\Rotation\decrypted_rotated2_frame_0372.png")
received_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f0\frame_0372.png")

# Calculate the BER
ber = calculate_ber(received_image, original_image)

# Print the BER
print("Bit Error Rate: {:.4f}".format(ber))


# def calculate_ber(received_image, original_image):
#     # Split the images into RGB planes
#     received_r, received_g, received_b = cv2.split(received_image)
#     original_r, original_g, original_b = cv2.split(original_image)

#     # Calculate the Hamming distance and BER for each RGB plane
#     ber_r = np.count_nonzero(received_r!= original_r) / received_r.size
#     ber_g = np.count_nonzero(received_g!= original_g) / received_g.size
#     ber_b = np.count_nonzero(received_b!= original_b) / received_b.size

#     return ber_r, ber_g, ber_b

# # Load the original and received images
# original_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\Rotation\decrypted_rotated2_frame_0001.png")
# received_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f3\frame_0001.png")

# # Calculate the BER for each RGB plane
# ber_r, ber_g, ber_b = calculate_ber(received_image, original_image)

# # Print the BER for each RGB plane
# print("Bit Error Rate (R): {:.4f}".format(ber_r))
# print("Bit Error Rate (G): {:.4f}".format(ber_g))
# print("Bit Error Rate (B): {:.4f}".format(ber_b))


Bit Error Rate: 0.9963


In [69]:
# PSNR on rotation attack

In [7]:
# clockwise

def MeanSquaredError(OriginalImage, ChangedImage):
        ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
        ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
        return ErrorValue

def PeakSignalNoiseRatio(OriginalImage, ChangedImage):
    ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
    ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
    if ErrorValue == 0:
        return 100
    ImageMaxPixelValue = 255.0
    PSNR = 10 * np.log10((ImageMaxPixelValue ** 2) / ErrorValue)
    return PSNR
if ChangedImage is not None:
    psnr = PeakSignalNoiseRatio(OriginalImage, ChangedImage)
    print("PSNR:", psnr)
else:
    print("Error: Failed to read ChangedImage")
OriginalImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\f0\frame_0029.png");
# ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\f3\frame_0029.png");
ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\Rotation\decrypted_rotated_frame_0029.png")
psnr=PeakSignalNoiseRatio(OriginalImage,ChangedImage);
print(psnr);

NameError: name 'ChangedImage' is not defined

In [8]:
# anti-clockwise
def MeanSquaredError(OriginalImage, ChangedImage):
        ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
        ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
        return ErrorValue

def PeakSignalNoiseRatio(OriginalImage, ChangedImage):
    ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
    ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
    if ErrorValue == 0:
        return 100
    ImageMaxPixelValue = 255.0
    PSNR = 10 * np.log10((ImageMaxPixelValue ** 2) / ErrorValue)
    return PSNR
OriginalImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\t0\frame_0372.png");
# ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\f3\frame_0029.png");
ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\Rotation\decrypted_rotated2_frame_0372.png")
psnr=PeakSignalNoiseRatio(OriginalImage,ChangedImage);
print(psnr);

AttributeError: 'NoneType' object has no attribute 'astype'

In [76]:
# FLIP ATTACK

In [62]:
import cv2
import numpy as np
import os

def perform_flip_attack(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all images in the input folder
    for filename in os.listdir(input_folder):
        # Read the encrypted image
        encrypted_image_path = os.path.join(input_folder, filename)
        encrypted_image = cv2.imread(encrypted_image_path)
        
        if encrypted_image is None:
            print(f"Error reading image: {encrypted_image_path}")
            continue
        
        # Flip attacks
        FA1 = cv2.flip(encrypted_image, -1)  # Flip Attack with both axes flipped
        FA2 = cv2.flip(encrypted_image, 1)   # Flip Attack with horizontal axis flipped
        FA3 = cv2.flip(encrypted_image, 0)   # Flip Attack with vertical axis flipped
        
        # Decrypt the images
        decrypted_image1 = decrypt_image(encrypted_image, FA1)
        decrypted_image2 = decrypt_image(encrypted_image, FA2)
        decrypted_image3 = decrypt_image(encrypted_image, FA3)
        
        # Save the flipped images
        cv2.imwrite(os.path.join(output_folder, f"Flipped1_{filename}"), FA1)
        cv2.imwrite(os.path.join(output_folder, f"Flippedminus1_{filename}"), FA2)
        cv2.imwrite(os.path.join(output_folder, f"Flipped0_{filename}"), FA3)
        
        # Save the decrypted images
        cv2.imwrite(os.path.join(output_folder, f"decryptedimage1_{filename}"), decrypted_image1)
        cv2.imwrite(os.path.join(output_folder, f"decryptedimage2_{filename}"), decrypted_image2)
        cv2.imwrite(os.path.join(output_folder, f"decryptedimage3_{filename}"), decrypted_image3)

        # print(f"Processed and saved images for: {filename}")

# Paths
input_folder = r"C:\Users\Asus\PYTHON\Images\t3"  # Path to the input folder of encrypted images
output_folder = r"C:\Users\Asus\PYTHON\Images\Flipattack"  # Path to save the processed images
# key = np.load("updated_key.npy")  # Path to the key file

# Perform flip attack and decryption on images in the input folder
perform_flip_attack(input_folder, output_folder)


In [83]:
# BER on flip attack

In [63]:
import numpy as np

def calculate_ber(received_image, original_image):
    # Calculate the Hamming distance between the two images
    hamming_distance = np.count_nonzero(received_image != original_image)

    # Calculate the number of pixels in the image
    num_pixels = received_image.size

    # Calculate the BER
    ber = hamming_distance / num_pixels

    return ber

# Load the original and received images
original_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\Flipattack\decryptedimage1_frame_0040.png")
received_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\t0\frame_0040.png")

# Calculate the BER
ber = calculate_ber(received_image, original_image)

# Print the BER
print("Bit Error Rate: {:.4f}".format(ber))

Bit Error Rate: 0.9961


In [79]:
# PSNR for flip attack

In [64]:
def MeanSquaredError(OriginalImage, ChangedImage):
        ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
        ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
        return ErrorValue

def PeakSignalNoiseRatio(OriginalImage, ChangedImage):
    ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
    ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
    if ErrorValue == 0:
        return 100
    ImageMaxPixelValue = 255.0
    PSNR = 10 * np.log10((ImageMaxPixelValue ** 2) / ErrorValue)
    return PSNR
OriginalImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\t0\frame_0032.png");
# ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\f3\frame_0029.png");
ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\Flipattack\decryptedimage1_frame_0032.png")
psnr=PeakSignalNoiseRatio(OriginalImage,ChangedImage);
print(psnr);

7.369981137642937


In [84]:
# Anti-Occlusion

In [77]:
import numpy as np

def calculate_ber(received_image, original_image):
    # Calculate the Hamming distance between the two images
    hamming_distance = np.count_nonzero(received_image != original_image)

    # Calculate the number of pixels in the image
    num_pixels = received_image.size

    # Calculate the BER
    ber = hamming_distance / num_pixels

    return ber

# Load the original and received images
original_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f3\frame_0050.png")
received_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f0\frame_0050.png")

# Calculate the BER
ber = calculate_ber(received_image, original_image)

# Print the BER
print("Bit Error Rate: {:.4f}".format(ber))

Bit Error Rate: 0.9961


In [86]:
# PSNR FOR ANTI-OCCLUSION

In [7]:
def MeanSquaredError(OriginalImage, ChangedImage):
        ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
        ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
        return ErrorValue

def PeakSignalNoiseRatio(OriginalImage, ChangedImage):
    ErrorValue = np.sum((OriginalImage.astype("float") - ChangedImage.astype("float")) ** 2)
    ErrorValue /= float(OriginalImage.shape[0] * OriginalImage.shape[1] * OriginalImage.shape[2])
    if ErrorValue == 0:
        return 100
    ImageMaxPixelValue = 255.0
    PSNR = 10 * np.log10((ImageMaxPixelValue ** 2) / ErrorValue)
    return PSNR
OriginalImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\rhino0\frame_0070.png");
# ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\f3\frame_0029.png");
ChangedImage=cv2.imread(r"C:\Users\Asus\PYTHON\Images\rhino2\frame_0070.png")
psnr=PeakSignalNoiseRatio(OriginalImage,ChangedImage);
print(psnr);

7.491382619032255


In [30]:
# antiocclusion 1/2

In [32]:
image_paths = [f"C:\\Users\\Asus\\PYTHON\\Images\\f3\\frame_{i:04d}.png" for i in range(29, 39)]

# Path to save the output images
save_path = "C:\\Users\\Asus\\PYTHON\\Images\\Anti-Occlusion-VipTrain\\images1_2"

for i, image_path in enumerate(image_paths):
    # Load the image
    image = cv2.imread(image_path)

    # Define the region of the image to apply the anti-occlusion attack (1/4 part of the image)
    height, width, channels = image.shape
    x = 0
    y = 0
    w = int(width/2)
    h = height
    
    # Ensure the region is within the image boundaries
    x = max(0, x)
    y = max(0, y)
    w = min(width, w)
    h = min(height, h)

    anti_occlusion_region = image[y:y+h, x:x+w]

    # Calculate the mean color of the surrounding area
    surrounding_area = image[max(0, y-50):min(height, y+h+50), max(0, x-50):min(width, x+w+50)]
    if surrounding_area.size > 0:
        mean_color = np.mean(surrounding_area, axis=(0, 1))
    else:
        mean_color = [0, 0, 0]  # default mean color if surrounding area is empty

    # Replace the occlusion with the mean color
    anti_occlusion_region = np.full_like(anti_occlusion_region, mean_color, dtype=np.uint8)

    # Combine the original and the modified part of the image
    image[y:y+h, x:x+w] = anti_occlusion_region

    # Save the image
    filename = f"frame_{i+29:04d}_anti_occlusion.png"
    cv2.imwrite(f"{save_path}\\{filename}", image)

    # Display the image
    # cv2.imshow(f'Anti-occlusion Attack Result {i+1}', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [21]:
def calculate_ber(received_image, original_image):
    # Calculate the Hamming distance between the two images
    hamming_distance = np.count_nonzero(received_image != original_image)

    # Calculate the number of pixels in the image
    num_pixels = received_image.size

    # Calculate the BER
    ber = hamming_distance / num_pixels

    return ber

# Load the original and received images
original_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f6\frame_0005.png")
received_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f1\frame_0005.png")

# Calculate the BER
ber = calculate_ber(received_image, original_image)

# Print the BER
print("Bit Error Rate: {:.4f}".format(ber))

Bit Error Rate: 0.3640


In [80]:
# Noise Attack Gaussain

In [89]:
def calculate_ber(original_image, decrypted_image):
    # Ensure images have the same dimensions
    if original_image.shape != decrypted_image.shape:
        raise ValueError("Original and decrypted images must have the same dimensions")

    # Convert images to grayscale if they are in color
    if len(original_image.shape) > 2:
        original_image = cv2.cvtColor(original_image)
    if len(decrypted_image.shape) > 2:
        decrypted_image = cv2.cvtColor(decrypted_image)

    # Calculate the number of differing bits
    error_bits = np.sum(original_image != decrypted_image)

    # Calculate total number of bits
    total_bits = original_image.size

    # Calculate Bit Error Rate (BER)
    ber = error_bits / total_bits

    return ber

def calculate_psnr(original_image, decrypted_image):
    mse = np.mean((original_image - decrypted_image) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 10 * np.log10(max_pixel **2 / (mse))
    return psnr

        #PSNR = 10 * np.log10((ImageMaxPixelValue ** 2) / ErrorValue)

# Load original and decrypted images
original_image = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f0\frame_0017.png",cv2.IMREAD_COLOR)
decrypted_image1 = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f6\frame_0017.png",cv2.IMREAD_COLOR)
decrypted_image2 = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f0\frame_0100.png",cv2.IMREAD_COLOR)
decrypted_image3 = cv2.imread(r"C:\Users\Asus\PYTHON\Images\f6\frame_0100.png",cv2.IMREAD_COLOR)


# Calculate BER
ber1 = calculate_ber(original_image, decrypted_image1)
ber2= calculate_ber(original_image, decrypted_image2)
ber3 = calculate_ber(original_image, decrypted_image3)
print("BER1:",ber1)
print("BER2:",ber2)
print("BER3:",ber3)

psnr1 = calculate_psnr(original_image, decrypted_image1)
psnr2 = calculate_psnr(original_image, decrypted_image2)
psnr3 = calculate_psnr(original_image, decrypted_image3)
print("PSNR1:", psnr1)
print("PSNR2:", psnr2)
print("PSNR3:", psnr3)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - cvtColor() missing required argument 'code' (pos 2)
>  - cvtColor() missing required argument 'code' (pos 2)
